5. 小程序常见的内存泄露问题
存在内存泄露问题会导致小程序在运行过程中内存占用持续增长，引起小程序闪退或被被微信强制销毁。

页面 unload 之后，基础库会从页面栈中将页面实例清理。正常情况下，JS 垃圾回收机制会将页面进行回收，释放内存。

但如果开发者代码中持有的页面实例（this）未释放，则会导致页面未被正常回收，引起内存泄露。建议开发者注意，并在 unload 中进行必要的清理。

案例一：页面实例被未解绑的事件监听引用

事件监听器中持有了页面的 this，如果页面销毁后监听未解绑，会导致页面无法释放。


```js
Page({
  themeChangeHandler({ theme }) {
    this.setData({ theme })
  },
  onLoad() {
    this._handler = this.themeChangeHandler.bind(this)
    wx.onThemeChange(this._handler)
  },
  // 修复方法：unload 中解绑监听
  // onUnload() {
  //   wx.offThemeChange(this._handler)
  // },
})
```


案例二：页面实例被页面外变量或全局变量引用

函数闭包内持有了页面的 this，且函数被挂到全局或页面声明周期外的变量，会导致页面无法释放。

```js
let languageListener = null

Page({
  onLoad() {
    getApp().userInfoChangeListener = ({ userName }) => {
      this.setData({ userName })
    }
    languageListener = ({ lang }) => {
      this.setData({ lang })
    }
  },
  // 修复方法：unload 中进行清理
  // onUnload() {
  //   getApp().userInfoChangeListener = null
  //   languageListener = null
  // },
})

```


### 案例三：页面实例被异步回调长时间引用

如果在长时间未返回的异步回调中访问了页面的 this，如持续时间过长的 setTimeout、setInterval，耗时较长的 wx API 回调（如长时间的 wx.request 等），会导致页面无法释放。

```js
Page({
  onLoad() {
    this._timer = setInterval(() => {
      this.setData({
        timerValue: Date.now()
      })
    }, 1000)
  },
  // 修复方法：unload 中进行清理
  // onUnload() {
  //   clearInterval(this._timer)
  // },
})
```

5.2 事件监听未及时解绑
事件监听结束后，应及时解绑监听器
```js


const locationChangeListener = function (res) {
  console.log('location change', res)
}
wx.onLocationChange(locationChangeListener)
wx.startLocationUpdate()
// 监听结束后
wx.stopLocationUpdate()
// 修复方法：不使用后及时解绑监听
// wx.offLocationChange(locationChangeListener)

```



